In [1]:
import json
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import cohere

# Inicializar Cohere
co = cohere.Client('OUxseEdjMfnqbmNkjQcNOcbLKkjFwHREtRJq72ZS')

# Cargar los datos desde el archivo JSON
with open('data.json', 'r', encoding='utf-8') as f:
    faqs = json.load(f)

# Extraer preguntas y respuestas
preguntas = [faq['pregunta'] for faq in faqs]
respuestas = [faq['respuesta'] for faq in faqs]


In [5]:
# Vectorizar las preguntas usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preguntas).toarray()

# Crear el índice FAISS
d = X.shape[1]
index = faiss.IndexFlatL2(d)
index.add(X)


In [6]:
def buscar_faq(query, k=5):
    vec_query = vectorizer.transform([query]).toarray()
    D, I = index.search(vec_query, k)
    return D[0], [respuestas[i] for i in I[0]], [preguntas[i] for i in I[0]]


In [7]:
def generar_respuesta_cohere(pregunta, umbral=1):
    # Buscar en el archivo JSON primero usando FAISS
    distancias, respuestas_similares, preguntas_similares = buscar_faq(pregunta)
    
    # Usar la respuesta similar si la distancia es menor que el umbral
    if distancias[0] < umbral:
        return respuestas_similares[0]
    
    # Construir el contexto con las preguntas y respuestas similares
    contexto = "\n".join([f"P: {preg}\nR: {resp}" for preg, resp in zip(preguntas_similares, respuestas_similares)])
    
    # Agregar la nueva pregunta al contexto
    pregunta_contexto = f"{contexto}\nP: {pregunta}\nR:"
    
    # Usar Cohere para generar una respuesta
    response = co.generate(
        prompt=pregunta_contexto,
        max_tokens=100,  # Permitir respuestas más largas
        stop_sequences=["\n"],  # Parar la generación en una nueva línea
        temperature=0.7,  # Aumentar la diversidad de la respuesta
    )
    return response.generations[0].text.strip()


In [8]:
query = "¿historia?"
print(generar_respuesta_cohere(query))


Debes tomar contacto con tu Director de Carrera para que canalice tu necesidad de cupos ante el Coordinador de Docencia o a su vez te informe cuales están disponible mismos que deben ser equivalentes.


In [9]:
import pickle

with open('modelo_faiss.pkl', 'wb') as f:
    pickle.dump({
        'index': index,
        'vectorizer': vectorizer,
        'respuestas': respuestas
    }, f)


In [10]:
import sys
import pickle
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import cohere

# Inicializar Cohere
co = cohere.Client('OUxseEdjMfnqbmNkjQcNOcbLKkjFwHREtRJq72ZS')

# Cargar los datos desde el archivo pickle
with open('modelo_faiss.pkl', 'rb') as f:
    data = pickle.load(f)
    index = data['index']
    vectorizer = data['vectorizer']
    respuestas = data['respuestas']

# Historial de conversación
conversation_history = []
print("Modelo cargado correctamente")


Modelo cargado correctamente


In [11]:
def buscar_faq(query, k=1):
    vec_query = vectorizer.transform([query]).toarray()
    D, I = index.search(vec_query, k)
    return D[0], [respuestas[i] for i in I[0]]

def generar_respuesta_cohere(pregunta, umbral=0.7):
    distancias, respuesta_similar = buscar_faq(pregunta)
    
    if distancias[0] < umbral:
        return respuesta_similar[0]
    
    # Construir el contexto de la conversación
    conversation = "Universidad de las Fuerzas Armadas ESPE:\n"
    for entry in conversation_history:
        conversation += f"{entry['role']}: {entry['message']}\n"
    conversation += f"Usuario: {pregunta}\nRespuesta:"
    
    response = co.generate(
        prompt=conversation,
        max_tokens=100,
        stop_sequences=["\n"],
        temperature=0.7,
    )
    
    return response.generations[0].text.strip()


In [14]:
query = "¿Quein es la directora de carrera?"
conversation_history.append({"role": "Usuario", "message": query})
respuesta = generar_respuesta_cohere(query, umbral=0.7)
conversation_history.append({"role": "AI", "message": respuesta})
print(conversation_history)
print(respuesta)


[{'role': 'Usuario', 'message': '¿Cuál es la misión de la ESPE?'}, {'role': 'AI', 'message': 'Al 2025, ser reconocidos a nivel nacional e internacional como una institución de educación superior de calidad en docencia, investigación y vinculación bajo el paradigma de una universidad inteligente, articulando la transferencia de ciencia y tecnología, a través de procesos de I+D+i; y, convirtiéndonos en un referente de pensamiento en seguridad y defensa, al servicio del país y Fuerzas Armadas.'}, {'role': 'Usuario', 'message': '¿Quien es el directo de carrerta?'}, {'role': 'AI', 'message': 'El dirigir del cargo es colaborar con las dependencias correspondientes para lograr mejores recursos para el personal y el mantenimiento de los requerimentos generales de lasareas de mando de la UNIDAD especializada de la ESPE.'}, {'role': 'Usuario', 'message': '¿Quein es la directora de carrera?'}, {'role': 'AI', 'message': 'Dirigir la carrera es asesorar y mantener vigentes y de calidad todas las car